In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random 
from sklearn.metrics import classification_report
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf
for dirname, _, filenames in os.walk('/content/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/kaggle/ner_dataset.csv.zip
/content/kaggle/ner.csv.zip


In [2]:
!unzip /content/kaggle/ner_dataset.csv.zip

Archive:  /content/kaggle/ner_dataset.csv.zip
  inflating: ner_dataset.csv         


In [3]:
!unzip /content/kaggle/ner.csv.zip

Archive:  /content/kaggle/ner.csv.zip
  inflating: ner.csv                 


In [4]:
data=pd.read_csv("/content/ner_dataset.csv",encoding="latin1")
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [5]:
data['Sentence #']=data['Sentence #'].ffill(axis = 0) 
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [6]:
agg_func = lambda s: [(w,p, t) for w,p, t in zip(s["Word"].values.tolist(),
                                                 s['POS'].values.tolist(),
                                                 s["Tag"].values.tolist())]

In [7]:
agg_data=data.groupby(['Sentence #']).apply(agg_func).reset_index().rename(columns={0:'Sentence_POS_Tag_Pair'})
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat..."
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s..."
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat..."
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O..."
4,Sentence: 10000,"[(U.N., NNP, B-geo), (relief, NN, O), (coordin..."


In [8]:
agg_data['Sentence']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[0] for s in sentence]))
agg_data['POS']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[1] for s in sentence]))
agg_data['Tag']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[2] for s in sentence]))

In [9]:
agg_data.shape

(47959, 5)

In [10]:
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair,Sentence,POS,Tag
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat...",Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s...",Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat...",Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O...",They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O
4,Sentence: 10000,"[(U.N., NNP, B-geo), (relief, NN, O), (coordin...",U.N. relief coordinator Jan Egeland said Sunda...,"NNP NN NN NNP NNP VBD NNP , NNP , JJ CC JJ JJ ...",B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [13]:
print(agg_data['Sentence'][0],'\t',agg_data['Tag'][0])

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country . 	 O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O


In [16]:
print(agg_data[0][0])

KeyError: ignored

In [14]:
agg_data['tokenised_sentences']=agg_data['Sentence'].apply(lambda x:x.split())
agg_data['tag_list']=agg_data['Tag'].apply(lambda x:x.split())
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair,Sentence,POS,Tag,tokenised_sentences,tag_list
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat...",Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s...",Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat...",Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O...",They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]"
4,Sentence: 10000,"[(U.N., NNP, B-geo), (relief, NN, O), (coordin...",U.N. relief coordinator Jan Egeland said Sunda...,"NNP NN NN NNP NNP VBD NNP , NNP , JJ CC JJ JJ ...",B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo..."


In [15]:
agg_data['len_sentence']=agg_data['tokenised_sentences'].apply(lambda x:len(x))
agg_data['len_tag']=agg_data['tag_list'].apply(lambda x:len(x))
agg_data['is_equal']=agg_data.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
agg_data['is_equal'].value_counts()

1    47955
0        4
Name: is_equal, dtype: int64

In [16]:
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair,Sentence,POS,Tag,tokenised_sentences,tag_list,len_sentence,len_tag,is_equal
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat...",Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...",24,24,1
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s...",Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...",25,25,1
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat...",Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...",32,32,1
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O...",They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]",11,11,1
4,Sentence: 10000,"[(U.N., NNP, B-geo), (relief, NN, O), (coordin...",U.N. relief coordinator Jan Egeland said Sunda...,"NNP NN NN NNP NNP VBD NNP , NNP , JJ CC JJ JJ ...",B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...",35,35,1


In [17]:
agg_data=agg_data[agg_data['is_equal']!=0]

In [18]:
agg_data.shape

(47955, 10)

In [19]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  47955
Are number of Sentences and Tag list equal  True


In [21]:
sentences_list[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [22]:
tags_list[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [23]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentences_list)

In [24]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  35170


In [25]:
tokeniser.index_word[326]

'national'

In [26]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

First Original Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Encoded Sentence  [1114, 4, 1161, 16, 1852, 230, 478, 6, 533, 1, 155, 5, 58, 8, 582, 1, 843, 4, 179, 87, 21, 15, 52, 2]
Is Length of Original Sentence Same as Encoded Sentence  True
Length of First Sentence  24


In [27]:
tags=list(set(data['Tag'].values))
print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

['B-tim', 'B-nat', 'I-art', 'I-gpe', 'I-nat', 'B-org', 'O', 'I-eve', 'I-org', 'B-per', 'I-per', 'B-geo', 'I-tim', 'I-geo', 'B-gpe', 'B-eve', 'B-art']
Number of Tags  17
Tags Map  {'B-tim': 0, 'B-nat': 1, 'I-art': 2, 'I-gpe': 3, 'I-nat': 4, 'B-org': 5, 'O': 6, 'I-eve': 7, 'I-org': 8, 'B-per': 9, 'I-per': 10, 'B-geo': 11, 'I-tim': 12, 'I-geo': 13, 'B-gpe': 14, 'B-eve': 15, 'B-art': 16}


In [28]:
reverse_tag_map={v: k for k, v in tags_map.items()}

In [29]:
print("Reverse Tags Map ", reverse_tag_map)

Reverse Tags Map  {0: 'B-tim', 1: 'B-nat', 2: 'I-art', 3: 'I-gpe', 4: 'I-nat', 5: 'B-org', 6: 'O', 7: 'I-eve', 8: 'I-org', 9: 'B-per', 10: 'I-per', 11: 'B-geo', 12: 'I-tim', 13: 'I-geo', 14: 'B-gpe', 15: 'B-eve', 16: 'B-art'}


In [30]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence ",sentences_list[0])
print('First Sentence Original Tags ',tags_list[0])
print("First Sentence Encoded Tags ",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence ",len(encoded_tags[0]))

First Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Sentence Original Tags  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']
First Sentence Encoded Tags  [6, 6, 6, 6, 6, 6, 11, 6, 6, 6, 6, 6, 11, 6, 6, 6, 6, 6, 14, 6, 6, 6, 6, 6]
Is length of Original Tags and Encoded Tags same  True
Length of Tags for First Sentence  24


In [31]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

104


In [32]:
max_len=128
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['O'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

Shape of Encoded Sentence  (47955, 128)
Shape of Encoded Labels  (47955, 128)
First Encoded Sentence Without Padding  [1114, 4, 1161, 16, 1852, 230, 478, 6, 533, 1, 155, 5, 58, 8, 582, 1, 843, 4, 179, 87, 21, 15, 52, 2]
First Encoded Sentence with padding  [1114    4 1161   16 1852  230  478    6  533    1  155    5   58    8
  582    1  843    4  179   87   21   15   52    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First Sentence Encoded Label without Padding  [6, 6, 6, 6, 6, 6, 11, 6, 6, 6, 6, 6, 11, 6, 6

In [33]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence  (128, 17)


In [34]:
target[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentences,target,test_size = 0.3,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_val_test,y_val_test,test_size = 0.2,random_state=42)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

Input Train Data Shape  (33568, 128)
Train Labels Length  33568
Input Test Data Shape  (2878, 128)
Test Labels Length  2878
Input Validation Data Shape  (11509, 128)
Validation Labels Length  11509


In [36]:
print("First Sentence in Training Data ",X_train[0])
print("First sentence Label ",y_train[0])
print("Shape of First Sentence -Train",X_train[0].shape)
print("Shape of First Sentence Label  -Train",y_train[0].shape)

First Sentence in Training Data  [ 144    1  873   20  198   13  132 1682    4    7  416 3636    5    1
 1396  653    2    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First sentence Label  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of First Sentence -Train (128,)
Shape of First Sentence Label  -Train (128, 17)


In [37]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional

In [50]:
embedding_dim=128
vocab_size=len(tokeniser.word_index)+1
lstm_units=128
max_len=128

input_word = Input(shape = (max_len,))
model = Embedding(input_dim = vocab_size+1,output_dim = embedding_dim,input_length = max_len)(input_word)
#model = Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat')(model)
model = LSTM(units=embedding_dim,return_sequences=True)(model)
out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 128, 128)          4501888   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128, 128)          131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 17)           2193      
Total params: 4,635,665
Trainable params: 4,635,665
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [52]:
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 32,epochs = 3)

Epoch 1/3
1049/1049 [==============================] - 38s 35ms/step - loss: 0.0998 - accuracy: 0.9809 - val_loss: 0.0366 - val_accuracy: 0.9910
Epoch 2/3
1049/1049 [==============================] - 35s 33ms/step - loss: 0.0255 - accuracy: 0.9930 - val_loss: 0.0248 - val_accuracy: 0.9927
Epoch 3/3
1049/1049 [==============================] - 34s 33ms/step - loss: 0.0176 - accuracy: 0.9946 - val_loss: 0.0225 - val_accuracy: 0.9932


In [53]:
preds=model.predict(X_test) ## Predict using model on Test Data

In [71]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_input']=test_data[i]
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]
    
    return pred_data,accuracy

In [72]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

Shape of Test Data Array (2878, 128)
Number of Test Data Points  2878


In [73]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [74]:
print(classification_report(y_actual,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        25
       B-eve       1.00      0.26      0.42        19
       B-geo       0.82      0.88      0.85      2190
       B-gpe       0.93      0.92      0.93       909
       B-nat       0.00      0.00      0.00         5
       B-org       0.70      0.62      0.66      1170
       B-per       0.83      0.82      0.83      1026
       B-tim       0.92      0.80      0.85      1244
       I-art       0.00      0.00      0.00        13
       I-eve       0.00      0.00      0.00        17
       I-geo       0.82      0.75      0.78       419
       I-gpe       1.00      0.28      0.43        18
       I-nat       0.00      0.00      0.00         1
       I-org       0.78      0.75      0.77       995
       I-per       0.92      0.84      0.88      1090
       I-tim       0.84      0.57      0.68       386
           O       0.98      0.99      0.99     53126

    accuracy              

In [75]:
pred_data[pred_data['actual_target_tag']=="B-art"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
9,9370,9370,Vioxx,16,5,B-art,B-org,27
11,3852,3852,Camp,16,5,B-art,B-org,223
7,25979,25979,alHurra,16,8,B-art,I-org,596
29,8157,8157,Please,16,6,B-art,O,690
31,8157,8157,Please,16,6,B-art,O,690
33,8157,8157,Please,16,6,B-art,O,690
4,7382,7382,Telegraph,16,11,B-art,B-geo,734
30,4008,4008,Nuclear,16,5,B-art,B-org,776
5,30307,30307,mon,16,6,B-art,O,858
5,20253,20253,Soprano,16,8,B-art,I-org,1550


In [76]:
pred_data[pred_data['actual_target_tag']=="B-nat"]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
11,950,950,Katrina,1,6,B-nat,O,637
6,4013,4013,Rita,1,6,B-nat,O,749
18,10325,10325,H5N2,1,6,B-nat,O,1508
18,10325,10325,H5N2,1,6,B-nat,O,1717
24,827,827,Hurricane,1,6,B-nat,O,2205


In [77]:
y_pred[0]

'O'

In [78]:
y_actual[0]

'O'

In [94]:
pred_data['tokens'][1]

1          top
1    spokesman
1         U.S.
1        Bowen
1        White
       ...    
1    companies
1          has
1         also
1           is
1         bomb
Name: tokens, Length: 2878, dtype: object

In [80]:
X_test[0]

array([   47,   252,    25,   151,   365,    17,  7063,  3479,  4053,
        7064,    72,     3, 32511,    23,     7,   493,  8195,     5,
         636,   104,     1,   120,  4937,   717,  5917,     2,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [81]:
test_sentences = tokeniser.sequences_to_texts(X_test)

In [130]:
test_sentences[0]

'A top U.S. official met with Nicaraguan President-elect Daniel Ortega Tuesday , hinting at a possible thaw in relations between the former Cold War enemies .'

In [83]:
y_test[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [84]:
y_pred[0]

'O'

In [85]:
preds[0]

array([[1.0620982e-04, 8.9750713e-05, 9.3903771e-05, ..., 3.4518129e-05,
        5.1193983e-05, 8.7045380e-05],
       [6.3192269e-06, 2.4061715e-06, 7.3615962e-07, ..., 1.3004379e-07,
        9.0533030e-07, 1.8238031e-06],
       [9.9398378e-05, 1.3288851e-04, 5.0997831e-05, ..., 2.2375599e-02,
        3.2329676e-04, 4.0299786e-04],
       ...,
       [2.6404052e-06, 2.3721473e-06, 1.1715245e-07, ..., 5.0174840e-08,
        4.0206746e-07, 8.4540466e-07],
       [2.6404052e-06, 2.3721473e-06, 1.1715245e-07, ..., 5.0174840e-08,
        4.0206746e-07, 8.4540466e-07],
       [2.6404025e-06, 2.3721473e-06, 1.1715245e-07, ..., 5.0174840e-08,
        4.0206746e-07, 8.4540466e-07]], dtype=float32)

In [86]:
pred_data['pred_target_index'][0]

0     6
0     6
0     6
0     9
0     6
     ..
0    14
0     9
0     9
0     6
0     6
Name: pred_target_index, Length: 2878, dtype: int64

In [87]:
pred_data['test_input'][0]

0      47
0       9
0      47
0      33
0      47
     ... 
0     272
0    6599
0    1626
0      61
0       9
Name: test_input, Length: 2878, dtype: int32

In [100]:
pred_data.shape

(62653, 8)

In [101]:
pred_data['test_input'].shape

(62653,)

In [97]:
X_test.shape

(2878, 128)

In [102]:
pred_data['id'][0]

0       1
0       2
0       3
0       4
0       5
     ... 
0    2874
0    2875
0    2876
0    2877
0    2878
Name: id, Length: 2878, dtype: int64

In [122]:
fin_data = pred_data.groupby('id')

In [124]:
fin_data.head()

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,47,47,A,6,6,O,O,1
1,252,252,top,6,6,O,O,1
2,25,25,U.S.,11,11,B-geo,B-geo,1
3,151,151,official,6,6,O,O,1
4,365,365,met,6,6,O,O,1
...,...,...,...,...,...,...,...,...
0,9,9,The,6,6,O,O,2878
1,161,161,bomb,6,6,O,O,2878
2,994,994,destroyed,6,6,O,O,2878
3,1,1,the,6,6,O,O,2878


In [131]:
pred_data[pred_data['id']==1]

,test_input,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,47,47,A,6,6,O,O,1
1,252,252,top,6,6,O,O,1
2,25,25,U.S.,11,11,B-geo,B-geo,1
3,151,151,official,6,6,O,O,1
4,365,365,met,6,6,O,O,1
5,17,17,with,6,6,O,O,1
6,7063,7063,Nicaraguan,14,14,B-gpe,B-gpe,1
7,3479,3479,President-elect,6,6,O,O,1
8,4053,4053,Daniel,9,9,B-per,B-per,1
9,7064,7064,Ortega,10,10,I-per,I-per,1
